# 01 Quick Tour
**本节目标**：

帮助读者快速掌握如何加载并使用模型推理

**备注**：

1. 本章节部分来自于Hugging Face的[官方文档](https://huggingface.co/docs/transformers/quicktour)以及[Generation with LLMS](https://huggingface.co/docs/transformers/llm_tutorial)，如果有进一步的需求可以参阅源文档以获取更加准确的知识。

2. 由于本教程关注LLM，因此会忽略官方文档Quick Tour中不相关的部分，并且本章节也包含一些简略的有关如何安装相关依赖的教程。

3. 本教程使用pytorch。

## 环境配置以及安装

在本部分中，我十分推荐使用conda的虚拟环境进行学习

*Terminal*

**新建环境**

`conda create --name LLM_Tutorial python=3.11`

**切换虚拟环境**

`conda activate LLM_Tutorial`

**安装ipykernel**

`conda install ipykernel`

In [3]:
!pip install transformers datasets
# 安装transformers以及datasets
# 如果你的源存在问题，无法直接安装，请按照教程切换源
# 临时方法 
# conda/pip install your_lib -i https://pypi.tuna.tsinghua.edu.cn/simple  清华源
# conda/pip install your_lib -i http://mirrors.aliyun.com/pypi/simple --trusted-host mirrors.aliyun.com  阿里源
# 配置方法请按你现在的环境进行教程搜索,这里不过多赘述

In [6]:
!pip3 install torch torchvision torchaudio
# 注意，此处直接安装torch并不可行，请访问 https://pytorch.org/ 根据自己环境的版本获取相应的pytorch安装命令进行安装

  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached MarkupSafe-2.1.3-cp311-cp311-macosx_10_9_universal2.whl.metadata (3.0 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 3.5 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 1.9 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 4.1 MB/s eta 0:00:0000:0100:01
Using cached networkx-3.2.1-py3-none-any.whl (1.6 MB)
Using cached MarkupSafe-2.1.3-cp311-cp311-macosx_10_9_universal2.whl (17 kB)


## 示例
<img src="./src/pytorch.png" alt="pytorch" width="1680"/>

例如我正在Mac上使用pip安装Python的Default版本，因此torch官方为我提供的命令是`pip3 install torch torchvision torchaudio`，在相应的开发机或本地上会有相应的命令。

## 模型&数据集的下载

由于一些原因，我们无法直接使用hugging face提供的类似于
`model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", device_map="auto")`
风格的代码直接获取hugging face上的模型，即使配置了系统代理也会因为不稳定导致下载途中断掉，因此我推荐我们使用*浏览器+VPN*的方式下载模型。

可以在[Tiny LLAMA](https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0/tree/main)处下载Tiny LLAMA的权重进行本教程的学习。

在本教程前期会尽可能的使用Tiny LLAMA进行讲解，后续涉及到相应的内容的时候会根据需要更换模型。

我们将下载好的模型存入`/model/Tiny-LLAMA`路径下

In [4]:
from transformers import AutoModelForCausalLM
import tqdm as notebook_tqdm

model = AutoModelForCausalLM.from_pretrained(
    "../model/Tiny-LLAMA"
)
# 加载模型到内存中

<img src="./src/error.png" alt="error" width="1680"/>

出现上述报错的主要原因是路径设置不同，可以检查自己IDE的路径设置，找到正确的模型存储位置解决

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("../model/Tiny-LLAMA", padding_side="left")
prompt = "<|system|>\nYou are a chatbot who can help code!</s>\n<|user|>\nWrite me a function to calculate the first 10 digits of the fibonacci sequence in Python and print it out to the CLI.</s>\n<|assistant|>"
model_inputs = tokenizer(prompt, return_tensors="pt")
# 加载一个prompt，然后tokenization为token id sequence

上述的prompt是Tiny-LLAMA官方给的example，让Tiny-LLAMA完成一个python语言的斐波那契数列结果计算代码

In [6]:
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

'<|system|>\nYou are a chatbot who can help code! \n<|user|>\nWrite me a function to calculate the first 10 digits of the fibonacci sequence in Python and print it out to the CLI. \n<|assistant|>\nHere\'s a Python function that calculates the first 10 digits of the Fibonacci sequence and prints them to the console:\n\n```python\ndef fibonacci_sequence(n):\n    """\n    Calculates the first n digits of the Fibonacci sequence.\n    """\n    if n <= 1:\n        return n\n    else:\n        return fibonacci_sequence(n-1) + fibonacci_sequence(n-2)\n\n# Example usage\nprint(fibonacci_sequence(10))  # Output: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]\n```\n\nTo use this function, simply call it with the desired number of digits:\n\n```\npython3 fibonacci_sequence_example.py 10\n```\n\nThis will print out the first 10 digits of the Fibonacci sequence.'

'<|system|>

You are a chatbot who can help code! 

<|user|>

Write me a function to calculate the first 10 digits of the fibonacci sequence in Python and print it out to the CLI. 

<|assistant|>

Here\'s a Python function that calculates the first 10 digits of the Fibonacci sequence and prints them to the console:
```python
def fibonacci_sequence(n):
    """Calculates the first n digits of the Fibonacci sequence."""
    if n <= 1:
        return n
    else:
        return fibonacci_sequence(n-1) + fibonacci_sequence(n-2)
        
    # Example usage\nprint(fibonacci_sequence(10))
    # Output: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
    
```
To use this function, simply call it with the desired number of digits.

This will print out the first 10 digits of the Fibonacci sequence.

官方的文档中也推荐了一种使用batch encode/decode进行加速的方法

In [7]:
tokenizer.pad_token = tokenizer.eos_token  # Most LLMs don't have a pad token by default
model_inputs = tokenizer(
    ["A list of colors: red, blue", "Portugal is"], return_tensors="pt", padding=True
)
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

['A list of colors: red, blue, green, yellow, purple, orange, brown, black, white, gray, pink, and yellow.\n\n2. Choose a font: Arial, Times New Roman, Georgia, Helvetica, or Comic Sans.\n\n3. Choose a background color: A light or dark gray, white, or black.\n\n4. Add a border: A thin black or white border around the image.\n\n5. Add a title: A title for the image, such as "My Favorite Color," "My Favorite Color Palette," or "My Favorite Colors."\n\n6. Add a caption: A brief description of the colors, such as "Red is my favorite color, and I love the vibrant hues of this palette."\n\n7. Add a link: A link to a website or blog post that provides more information about the colors.\n\n8. Add a call-to-action: A call-to-action, such as "Share your favorite color palette on social media using the hashtag #MyFavoriteColorPalette."\n\n9. Add a footer: A footer with the website\'s name, address, and phone number.\n\n10. Save and publish: Save the image and publish it on your website or blog.',

至此我们已经搞定了一个LLM的基础环境配置，下载，加载与最简单的推理